<a href="https://colab.research.google.com/github/student-prpotecollege/sampledemo/blob/main/FOODdELIVERY_ANALYSIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [137]:
import pandas as pd
import sqlite3


In [138]:
orders = pd.read_csv("orders.csv")
orders.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [139]:
users = pd.read_json("users.json")
users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [140]:
conn = sqlite3.connect(":memory:")

with open("restaurants.sql", "r", encoding="utf-8") as f:
    conn.executescript(f.read())

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [141]:
final_df = orders.merge(users, on="user_id", how="left") \
                 .merge(restaurants, on="restaurant_id", how="left")

final_df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [142]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)



In [143]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_csv('final_food_delivery_dataset.csv')

# Convert order_date to datetime objects immediately for accurate analysis later
df['order_date'] = pd.to_datetime(df['order_date'], dayfirst=True)

# Quick sanity check of the data
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   order_id           10000 non-null  int64         
 1   user_id            10000 non-null  int64         
 2   restaurant_id      10000 non-null  int64         
 3   order_date         10000 non-null  datetime64[ns]
 4   total_amount       10000 non-null  float64       
 5   restaurant_name_x  10000 non-null  object        
 6   name               10000 non-null  object        
 7   city               10000 non-null  object        
 8   membership         10000 non-null  object        
 9   restaurant_name_y  10000 non-null  object        
 10  cuisine            10000 non-null  object        
 11  rating             10000 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(6)
memory usage: 937.6+ KB
None


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,2023-02-18,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,2023-01-18,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,2023-07-15,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,2023-10-04,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,2023-12-25,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [144]:
print("--- Section 1 Answers ---")
# 1. Gold Membership Orders
gold_orders = df[df['membership'] == 'Gold']
print(f"Total Gold orders: {len(gold_orders)}")

# 2. Revenue from Hyderabad
hyd_revenue = df[df['city'] == 'Hyderabad']['total_amount'].sum()
print(f"Total Revenue from Hyderabad: {round(hyd_revenue)}")

# 3. Distinct Users
print(f"Distinct Users: {df['user_id'].nunique()}")

# 4. Average Order Value (Gold)
avg_gold_val = gold_orders['total_amount'].mean()
print(f"Average Order Value (Gold): {round(avg_gold_val, 2)}")

# 5. High Rated Orders (>= 4.5)
high_rated_orders = df[df['rating'] >= 4.5]
print(f"Orders with Rating >= 4.5: {len(high_rated_orders)}")

# 6. Top City for Gold Orders
# Group by city to find the one with highest Gold revenue
gold_city_revenue = gold_orders.groupby('city')['total_amount'].sum()
top_gold_city = gold_city_revenue.idxmax()
top_city_count = len(gold_orders[gold_orders['city'] == top_gold_city])
print(f"Orders in Top Revenue City ({top_gold_city}) for Gold Members: {top_city_count}")

--- Section 1 Answers ---
Total Gold orders: 4987
Total Revenue from Hyderabad: 1889367
Distinct Users: 2883
Average Order Value (Gold): 797.15
Orders with Rating >= 4.5: 3374
Orders in Top Revenue City (Chennai) for Gold Members: 1337


In [145]:
print("--- Section 2 Answers ---")
print("1. Join Column: user_id")
print("2. Rating/Cuisine File Format: JSON")
print(f"3. Total Rows: {len(df)}")
print("4. Missing Values Fill: NaN")
print("5. Merge Function: merge")
print("6. Membership Source: users.json")
print("7. Join Key (Restaurants): restaurant_id")
print("8. Food Type Column: cuisine")
print("9. User Details Repetition: multiple times")

--- Section 2 Answers ---
1. Join Column: user_id
2. Rating/Cuisine File Format: JSON
3. Total Rows: 10000
4. Missing Values Fill: NaN
5. Merge Function: merge
6. Membership Source: users.json
7. Join Key (Restaurants): restaurant_id
8. Food Type Column: cuisine
9. User Details Repetition: multiple times


In [146]:
# Q1: Highest Gold Revenue City
# We calculated 'gold_city_revenue' in Section 1
print(f"1. City with highest Gold revenue: {gold_city_revenue.idxmax()}")

# Q2: Highest AOV Cuisine
cuisine_aov = df.groupby('cuisine')['total_amount'].mean()
print(f"2. Cuisine with highest AOV: {cuisine_aov.idxmax()}")

# Q3: High Value Users (>1000 total)
user_lifetime_value = df.groupby('user_id')['total_amount'].sum()
high_value_users = (user_lifetime_value > 1000).sum()
print(f"3. Users with >1000 total value: {high_value_users}")

# Q4: Revenue by Rating Range
# Binning the ratings to see which range is most profitable
rating_bins = pd.cut(df['rating'], bins=[3.0, 3.5, 4.0, 4.5, 5.0], labels=['3.0-3.5', '3.6-4.0', '4.1-4.5', '4.6-5.0'])
print("4. Revenue by Rating Range:")
print(df.groupby(rating_bins)['total_amount'].sum().sort_values(ascending=False))

# Q5: Highest AOV City (Gold)
gold_city_aov = gold_orders.groupby('city')['total_amount'].mean()
print(f"5. City with highest Gold AOV: {gold_city_aov.idxmax()}")

# Q6: Least Distinct Restaurants
distinct_rest_count = df.groupby('cuisine')['restaurant_id'].nunique()
print(f"6. Cuisine with fewest restaurants: {distinct_rest_count.idxmin()}")

# Q7: Gold Order Percentage
gold_pct = (len(gold_orders) / len(df)) * 100
print(f"7. Gold Order Percentage: {round(gold_pct)}%")

# Q8: High AOV but Low Volume Restaurant
# Filtering for restaurants with < 20 orders
rest_stats = df.groupby('restaurant_name_x').agg({'total_amount': 'mean', 'order_id': 'count'})
target_rest = rest_stats[rest_stats['order_id'] < 20].sort_values('total_amount', ascending=False).head(1)
print(f"8. Target Restaurant: {target_rest.index[0]}")

# Q9: Best Revenue Combination
combo_revenue = df.groupby(['membership', 'cuisine'])['total_amount'].sum()
print(f"9. Top Revenue Combo: {combo_revenue.idxmax()}")

# Q10: Best Quarter
df['quarter'] = df['order_date'].dt.quarter
quarter_revenue = df.groupby('quarter')['total_amount'].sum()
print(f"10. Best Revenue Quarter: Q{quarter_revenue.idxmax()}")


1. City with highest Gold revenue: Chennai
2. Cuisine with highest AOV: Mexican
3. Users with >1000 total value: 2544
4. Revenue by Rating Range:
rating
4.6-5.0    2197030.75
4.1-4.5    1960326.26
3.0-3.5    1881754.57
3.6-4.0    1717494.41
Name: total_amount, dtype: float64
5. City with highest Gold AOV: Chennai
6. Cuisine with fewest restaurants: Chinese
7. Gold Order Percentage: 50%
8. Target Restaurant: Hotel Dhaba Multicuisine
9. Top Revenue Combo: ('Regular', 'Mexican')
10. Best Revenue Quarter: Q3


/tmp/ipython-input-3292588851.py:18: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(df.groupby(rating_bins)['total_amount'].sum().sort_values(ascending=False))
